<a href="https://colab.research.google.com/github/kapooraditi79/RAG_Projects/blob/main/bing_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Referenced Sites:
* https://docling-project.github.io/docling/examples/hybrid_chunking/#overview

* https://www.youtube.com/watch?v=9lBTS5dM27c&list=WL&index=2&t=1s

* Contextual Chunking-> https://www.anthropic.com/news/contextual-retrieval

* LanceDB embeddings-> https://lancedb.github.io/lancedb/embeddings/default_embedding_functions/#text-embedding-functions

I am doing text embeddings only. Not multi-modal embeddings.

* Building with Google's gemini embeddings-> https://lancedb.github.io/lancedb/embeddings/available_embedding_models/text_embedding_functions/gemini_embedding/

In [ ]:
! pip install docling

In [10]:
from docling.document_converter import DocumentConverter
converter= DocumentConverter()

result= converter.convert('combined_markdown.md')
result= result.document

In [ ]:
! pip install huggingface_hub

In [2]:
from huggingface_hub import login
from google.colab import userdata
token= userdata.get('HF_TOKEN_ORIGINAL_AGENTCOURSE')
login(token=token)

Setting up the configurations of bitsAndBytes config.

Note-> the latest version ,when i made this notebook, of bitsandbytes==0.46 is supported by only cuda==12.3. while the latest version of cuda is 12.4.
So i made the following changes. make these changes only after a proper research.

In [ ]:
# 1. Clean broken bitsandbytes installs
!pip uninstall -y bitsandbytes
!rm -rf /usr/local/lib/python*/dist-packages/bitsandbytes*

# 2. Set the manual override for CUDA 12.3
import os
os.environ["BNB_CUDA_VERSION"] = "123"

# 3. Reinstall the latest bitsandbytes (supports 12.3)
!pip install bitsandbytes
!pip install -U transformers accelerate

# 4. Verify CUDA backend
!python -m bitsandbytes


In [4]:
# loading the model for tokenizing

from transformers import BitsAndBytesConfig, Gemma3ForCausalLM

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = Gemma3ForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    quantization_config=bnb_config,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

This can be used to load a bitsandbytes version built with a CUDA version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=



model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [5]:
# let us work on the chunking using docling
!pip install -qU pip docling transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.8 MB/s eta 0:00:00


In [ ]:
# if you want to use the hugging face tokenizers
!pip install 'docling-core[chunking]'

In RAG it is important to make sure that the chunker and the embedding model are using the same tokenizer.

okay so currently i do not see

In [7]:
# lets build our tokenizer
from docling_core.transforms.chunker.tokenizer.huggingface import HuggingFaceTokenizer
from transformers import AutoTokenizer
model_id= "google/gemma-3-1b-it"

max_tokens= 400

tokenizer= HuggingFaceTokenizer(
    tokenizer=AutoTokenizer.from_pretrained(model_id),
    max_tokens=max_tokens
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [8]:
# now we can build our chunker
from docling.chunking import HybridChunker

chunker= HybridChunker(
    tokenizer=tokenizer,
    merge_peers= True # merges the undersized chunks that share the same headings or metadata
)

def build_and_get_chunks(doc):
  chunk_iter= chunker.chunk(doc)
  chunks= list(chunk_iter)

  return chunks

In [11]:
chunks= build_and_get_chunks(result)

we want to embed the contextualized chunkings. so:

In [12]:
def get_contextualized_chunks(chunks):
  contextualized_chunks= []
  for chunk in chunks:
    enriched_text= chunker.contextualize(chunk=chunk)
    contextualized_chunks.append(enriched_text)
  return contextualized_chunks

In [13]:
contextualized_chunk_list= get_contextualized_chunks(chunks)

In [15]:
contextualized_chunk_list[100]

"Dear\tRon,\tand\tHarry\tif\tyou're\tthere,\nDraco\tturned\taway\tand\tsaw\tthe\tcabinet\tright\tin\tfront\tof\thim.\tHe\twalked forward\t…\the\tstretched\tout\this\thand\tfor\tthe\thandle\t…\n'Done,'\tsaid\tMr\tMalfoy\tat\tthe\tcounter.\t'Come,\tDraco!'\nHarry\twiped\this\tforehead\ton\this\tsleeve\tas\tDraco\tturned\taway.\n'Good\tday\tto\tyou,\tMr\tBorgin,\tI'll\texpect\tyou\tat\tthe\tmanor\ttomorrow\tto pick\tup\tthe\tgoods.'\nThe\tmoment\tthe\tdoor\thad\tclosed,\tMr\tBorgin\tdropped\this\toily\tmanner.\n'Good\tday\tyourself, Mister Malfoy,\tand\tif\tthe\tstories\tare\ttrue,\tyou\thaven't sold\tme\thalf\tof\twhat's\thidden\tin\tyour manor …'\nMuttering\tdarkly,\tMr\tBorgin\tdisappeared\tinto\ta\tback\troom.\tHarry\twaited for\ta\tminute\tin\tcase\the\tcame\tback,\tthen,\tquietly\tas\the\tcould,\tslipped\tout\tof\tthe cabinet,\tpast\tthe\tglass\tcases\tand\tout\tof\tthe\tshop\tdoor."

In [16]:
! pip install -U -q 'google-genai'

In [17]:
from google import genai

client=genai.Client(api_key=userdata.get('GOOGLE_2_API_KEY'))

In [ ]:
! pip install chromadb

In [25]:
from google.genai import types
from chromadb import Documents, EmbeddingFunction, Embeddings

class GeminiEmbeddingFunction(EmbeddingFunction):
  def __init__(self):
    pass
    # required by chromadb


  def __call__(self, input:Documents) -> Embeddings:
    Embedding_Model_ID= "models/text-embedding-004"

    # ensuring input is formatted
    if isinstance(input, str):
      input= [input]

    all_embeddings=[]
    batch_size=100

    for i in range(0, len(input), batch_size):
      batch= input[i:i+batch_size]
      try:
        response= client.models.embed_content(
            model=Embedding_Model_ID,
            contents= input,
            config=types.EmbedContentConfig(
                task_type='retrieval_document',

            )
        )
        # return all teh embeddings
        batch_embeddings= [emb.values for emb in response.embeddings]
        all_embeddings.extend(batch_embeddings)

      except Exception as e:
        print(f"Embedding error:{e}")
        return None
    return all_embeddings

In [26]:
# fix chromadb for batch processing
import chromadb
def create_chroma_db(contextualized_chunk_list):
  chroma_client= chromadb.PersistentClient(path="novel_collection_embeddings_3")

  collection= chroma_client.get_or_create_collection(
      name='bing_novelrag_collection_3',
      embedding_function=GeminiEmbeddingFunction()
  )
  print(f"Adding {len(contextualized_chunk_list)} chunks to database...")

  # add documents in batches to avoid memory issues
  batch_size=50

  for i in range(0, len(contextualized_chunk_list), batch_size):
    end_idx= min(i+batch_size, len(contextualized_chunk_list))
    batch_docs= contextualized_chunk_list[i:end_idx]
    batch_ids= [str(j) for j in range(i, end_idx)]


    collection.add(
        documents= batch_docs,
        ids= batch_ids
    )

    print(f"Added batch {i//batch_size + 1}/{(len(contextualized_chunk_list)+ batch_size - 1)//batch_size}")


  print(f"Database created with {collection.count()} documents")

  return collection


In [27]:
# Set up the DB
collection_built = create_chroma_db(contextualized_chunk_list)

Adding 1346 chunks to database...
Added batch 1/27
Added batch 2/27
Added batch 3/27
Added batch 4/27
Added batch 5/27
Added batch 6/27
Added batch 7/27
Added batch 8/27
Added batch 9/27
Added batch 10/27
Added batch 11/27
Added batch 12/27
Added batch 13/27
Added batch 14/27
Added batch 15/27
Added batch 16/27
Added batch 17/27
Added batch 18/27
Added batch 19/27
Added batch 20/27
Added batch 21/27
Added batch 22/27
Added batch 23/27
Added batch 24/27
Added batch 25/27
Added batch 26/27
Added batch 27/27
Database created with 1346 documents


In [28]:
collection_built.peek()

{'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'embeddings': array([[-0.03861213,  0.02344161, -0.00374704, ...,  0.00964072,
          0.0244747 , -0.01426351],
        [-0.02072495,  0.02093361, -0.00411656, ...,  0.05033953,
         -0.00417255, -0.02308157],
        [-0.0383333 ,  0.01372326, -0.02511429, ...,  0.06165662,
          0.01992766,  0.00818526],
        ...,
        [-0.03405032,  0.00664998, -0.00151566, ...,  0.0002513 ,
          0.04247424, -0.03012509],
        [-0.05485309, -0.00544187,  0.01287586, ..., -0.02370497,
          0.02652791, -0.02154174],
        [-0.03307312, -0.00749543, -0.02514764, ...,  0.01036208,
         -0.01646801, -0.03018571]]),
 'documents': ['HARRY POTTER\nand\tthe\tChamber\tof\tSecrets\nJ.K.\tROWLING\nAll\trights\treserved;\tno\tpart\tof\tthis\tpublication\tmay\tbe\treproduced\tor\ttransmitted\tby\tany\tmeans, electronic,\tmechanical,\tphotocopying\tor\totherwise,\twithout\tthe\tprior\tpermission\tof\tthe\tpublisher\nTh

In [29]:
def get_relevant_passage(query, db, n_results=3):
  """Get relevant passages with better error handling"""

  try:
    results=db.query(query_texts=[query], n_results=n_results)
    if not results['documents']:
      print("No results found")
      return None
    return results['documents'][0]

  except Exception as e:
    print(f"Error in query: {e}")
    return None

In [31]:
# Perform embedding search
passage = get_relevant_passage("where does harry potter go to school?", collection_built)
print(passage)

['CURRENTLY HEADMASTER OF HOGWARTS\n"Are you sure that\'s a real spell?" said the girl. "Well, it\'s not very good, is it? I\'ve tried a few simple spells just for practice and it\'s all worked for me. Nobody in my family\'s magic at all, it was ever such a surprise when I got my letter, but I was ever so pleased, of course, I mean, it\'s the very best school of witchcraft there is, I\'ve heard -I\'ve learned all our course books by heart, of course, I just hope it will be enough -- I\'m Hermione Granger, by the way, who are you.\nShe said all this very fast.\nHarry looked at Ron, and was relieved to see by his stunned face that he hadn\'t learned all the course books by heart either.\n"I\'m Ron Weasley," Ron muttered.\n"Harry Potter," said Harry.\n"Are you really?" said Hermione. "I know all about you, of course -- I got a few extra books. for background reading, and you\'re in Modern Magical History and The Rise and Fall of the Dark Arts and Great Wizarding Events of the Twentieth Ce